In [7]:
! curl "https://raw.githubusercontent.com/PerseusDL/canonical-greekLit/master/data/tlg0012/tlg001/tlg0012.tlg001.perseus-eng3.xml" -o eng.xml

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1107k  100 1107k    0     0   801k      0  0:00:01  0:00:01 --:--:--  802k


In [8]:
! curl "https://raw.githubusercontent.com/PerseusDL/canonical-greekLit/master/data/tlg0012/tlg001/tlg0012.tlg001.perseus-grc2.xml" -o grc.xml

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2012k  100 2012k    0     0  2082k      0 --:--:-- --:--:-- --:--:-- 2080k


First the Greek text:

In [3]:
import xml.etree.ElementTree as ET
tree = ET.parse('grc.xml')
root = tree.getroot()
books = {}
for book in tree.findall(".//*[@subtype='Book']"):
    lines = {}
    line_els = book.findall('.//{*}l')
    for l in line_els:
        lines[int(l.attrib["n"])] = "".join(l.itertext()).strip()
    books[int(book.attrib["n"])] = lines
    #print(elem.get("n"))

In [215]:
books[1][2]

'οὐλομένην, ἣ μυρίʼ Ἀχαιοῖς ἄλγεʼ ἔθηκε,'

Then the English text:

In [8]:
def parseCard(root, tr_lines = {}, cur = 0, cur_text = ""):
    for item in root:
        if item.text and item.tag != "{http://www.tei-c.org/ns/1.0}note":
            cur_text += item.text
        if item.tag == "{http://www.tei-c.org/ns/1.0}milestone":
            if cur: tr_lines[cur] = cur_text.strip() + ' '
            cur_text = ""
            new_cur = int(item.attrib["n"])
            cur = new_cur + 1 if new_cur != 1 else 1
        cur_text, cur, tr_lines = parseCard(item, tr_lines, cur, cur_text)
        if item.tail:
            cur_text += item.tail.strip() + ' '
    return (cur_text, cur, tr_lines)


In [9]:
import xml.etree.ElementTree as ET

tree = ET.parse('eng.xml')
root = tree.getroot()
tr_books = {}
for book in tree.findall(".//*[@subtype='book']"):
    card_els = book.findall(".//*[@subtype='card']")
    tr_lines = {}
    for l in card_els:
        cur = int(l.attrib["n"])
        cur_text, cur, tr_lines = parseCard(l, tr_lines, cur)
        tr_lines[cur] = cur_text.strip()
    tr_books[int(book.attrib["n"])] = tr_lines.copy()
    #print(elem.get("n"))
print(tr_books[1][186])


how much mightier I am than you, and another may shrink from declaring himself my equal and likening himself to me to my face. So he spoke. Grief came upon the son of Peleus, and within his shaggy breast his heart was divided, whether he should draw his sharp sword from beside his thigh, 


In [44]:
import base64
output_dir = "../translation/"

for k, book in books.items():
    tr_book = tr_books[k]

    output_file = open(output_dir + f"translation{k}.js", 'w')
    output_file.write("var translation = {\n")
    
    pi = 0
    lines_in_book = list(book.keys())
    for i in range(1, max(lines_in_book)+1):
        if i not in tr_book:
            output_file.write(f"{i}: {pi},\n")
            continue
        tr = base64.b64encode(bytes(tr_book[i], 'utf-8')).decode("ascii")
        output_file.write(f"{i}: '{tr}',\n")
        pi = i
    output_file.write("}\n")


In [ ]:
return base64.b64encode(ET.tostring(translation, encoding="unicode").encode()).decode("ascii") 


In [99]:
import re

lines = [l for l in [books["1"][str(i)] for i in range(33,68)]]
breaks = [0] + [i+1 for i,l in enumerate(lines) if any(x in ['·',';','.'] for x in l)]
gr_segments = []
for i in range(0, len(breaks)-1):
    f = breaks[i]
    t = breaks[i+1]
    gr_segments += ["\n".join(lines[f:t])]
tr_segments = re.findall('.*?[.\.\?;]', tr_books["1"]["33"])
for i,s in enumerate(gr_segments):
    print(s)
    print('\n')
    print(tr_segments[i])
    print('-----------------------')


ὣς ἔφατʼ, ἔδεισεν δʼ ὃ γέρων καὶ ἐπείθετο μύθῳ·


So he spoke, and the old man was seized with fear and obeyed his word.
-----------------------
βῆ δʼ ἀκέων παρὰ θῖνα πολυφλοίσβοιο θαλάσσης·


So he spoke in prayer, and Phoebus Apollo heard him.
-----------------------
πολλὰ δʼ ἔπειτʼ ἀπάνευθε κιὼν ἠρᾶθʼ ὃ γεραιὸς
Ἀπόλλωνι ἄνακτι, τὸν ἠΰκομος τέκε Λητώ·


  Down from the peaks of Olympus he strode, angered at heart, bearing on his shoulders his bow and covered quiver.
-----------------------
κλῦθί μευ ἀργυρότοξʼ, ὃς Χρύσην ἀμφιβέβηκας
Κίλλαν τε ζαθέην Τενέδοιό τε ἶφι ἀνάσσεις,
Σμινθεῦ εἴ ποτέ τοι χαρίεντʼ ἐπὶ νηὸν ἔρεψα,
ἢ εἰ δή ποτέ τοι κατὰ πίονα μηρίʼ ἔκηα
ταύρων ἠδʼ αἰγῶν, τὸ δέ μοι κρήηνον ἐέλδωρ·


The arrows rattled on the shoulders of the angry god as he moved, and his coming was like the night.
-----------------------
τίσειαν Δαναοὶ ἐμὰ δάκρυα σοῖσι βέλεσσιν.


  Then he sat down apart from the ships and let fly an arrow: terrible was the twang of the silver bow.
-------------

IndexError: list index out of range